<a href="https://colab.research.google.com/github/Vonewman/Deep_Learning_models_with_pytorch/blob/master/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### 0) Prepare data

In [3]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [4]:
n_samples, n_features = X.shape

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1234)

In [6]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# 1) Model
# Linear model f = wx + b, sigmoid at the end

In [8]:
class Model(nn.Module):

    def __init__(self, n_input_features):

        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

model = Model(n_features)

### 2) Loss and Optimizer

In [9]:
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 3) Training loop

In [10]:
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass ans update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")

epoch: 10, loss = 0.6289
epoch: 20, loss = 0.5167
epoch: 30, loss = 0.4446
epoch: 40, loss = 0.3942
epoch: 50, loss = 0.3568
epoch: 60, loss = 0.3279
epoch: 70, loss = 0.3046
epoch: 80, loss = 0.2855
epoch: 90, loss = 0.2694
epoch: 100, loss = 0.2556


In [11]:
# test the data
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')


accuracy: 0.6316
